In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import math
import requests
import scipy.stats as stats
import datetime as dt

In [8]:
sr = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/Andre15.csv")
hrn = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/hrnmaster702.csv")
ped = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/pedmaster702.csv")
#df = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/sdperformance701.csv")

unwanted = hrn.columns[hrn.columns.str.startswith('Unnamed:')]
hrn.drop(unwanted, axis=1, inplace=True)
print(hrn.shape)
hrn2 = hrn.drop_duplicates(subset=['Horse','Date'],keep='first')
print(hrn2.shape)

unwanted = ped.columns[ped.columns.str.startswith('Unnamed:')]
ped.drop(unwanted, axis=1, inplace=True)
ped = ped.drop_duplicates(keep='first')
print(ped.shape)
ped2 = ped[~ped['Name'].duplicated(keep=False) | ped[['S']].notnull().any(axis=1)]
print(ped2.shape)

(473626, 12)
(466485, 12)
(102258, 15)
(101989, 15)


In [9]:
hrn['Date']
substring = 'R'
dde = list(hrn['Date'])

datelist = list()
for i in list(hrn['Date']):
    if i is np.nan:
        datelist.append(np.nan)    
    elif substring in i:
        datelist.append(i.lstrip().split('-')[0])
    else:
        datelist.append(i)

hrn['Datecol'] = datelist

hrn['Datecol'] = pd.to_datetime(hrn["Datecol"])
sr['OBSDate'] = pd.to_datetime(sr['OBSDate'])

adts = list(sr['OBSDate'].unique())
adtsmo = list()
adts2 = list()
for d in adts:
    adtsmo.append(pd.to_datetime(d) +  pd.DateOffset(months = 1))
for d in adts:
    adts2.append(pd.to_datetime(d))

col = list()
for d in adts2:
    col.append(str(d.month)+ '-' + str(d.year))
    
cold = [(adts2[i], col[i]) for i in range(0, len(adts))]
cold2 = {col[i]: adts2[i] for i in range(len(col))}

Transform race results

In [10]:
rr = hrn

rr['Grade'] = hrn['Race'].str[-2:]
rr['count'] = 1
dummies = pd.get_dummies(rr['Grade'])
rr1 = pd.concat([rr, dummies[['G1','G2','G3','ow']]], axis=1)
rr1['first'] = np.where((rr1['Fin'] == '1st') | (rr1['Fin'] == '1'),1,0)
rr1['second'] = np.where((rr1['Fin'] == '2nd') | (rr1['Fin'] == '2'),1,0)
rr1['third'] = np.where((rr1['Fin'] == '3rd') | (rr1['Fin'] == '3'),1,0)
rr1['MCl'] = np.where(rr1[['G1','G2','G3','ow']].all(axis=1), 0, 1)
rr2 = rr1

for c in cold:
    rr2[c[1]] = rr2['Datecol'] < c[0]
    rr2[c[1] + ':G1'] = (rr2['G1'].fillna(0) * 5) * (rr2['first'] * 3) * rr2[c[1]]
    rr2[c[1] + ':G1_2'] = (rr2['G1'].fillna(0) * 5) * (rr2['second'] * 2) * rr2[c[1]]
    rr2[c[1] + ':G1_3'] = (rr2['G1'].fillna(0) * 5) * (rr2['third'] * 1) * rr2[c[1]]
    
    rr2[c[1] + ':G2'] = (rr2['G2'].fillna(0) * 4) * (rr2['first'] * 3) * rr2[c[1]]
    rr2[c[1] + ':G2_2'] = (rr2['G2'].fillna(0) * 4) * (rr2['second'] * 2) * rr2[c[1]]
    rr2[c[1] + ':G2_3'] = (rr2['G2'].fillna(0) * 4) * (rr2['third'] * 1) * rr2[c[1]]
    
    rr2[c[1] + ':G3'] = (rr2['G3'].fillna(0) * 3) * (rr2['first'] * 3) * rr2[c[1]]
    rr2[c[1] + ':G3_2'] = (rr2['G3'].fillna(0) * 3) * (rr2['second'] * 2) * rr2[c[1]]
    rr2[c[1] + ':G3_3'] = (rr2['G3'].fillna(0) * 3) * (rr2['third'] * 1) * rr2[c[1]]
    
    rr2[c[1] + ':Al'] = (rr2['ow'].fillna(0) * 2) * (rr2['first'] * 3) * rr2[c[1]]
    rr2[c[1] + ':Al_2'] = (rr2['ow'].fillna(0) * 2) * (rr2['second'] * 2) * rr2[c[1]]
    rr2[c[1] + ':Al_3'] = (rr2['ow'].fillna(0) * 2) * (rr2['third'] * 1) * rr2[c[1]]
        
    rr2[c[1] + ':MCl'] = (rr2['MCl']) * (rr2['first'] * 3) * rr2[c[1]]
    rr2[c[1] + ':MCl_2'] = (rr2['MCl']) * (rr2['second'] * 2) * rr2[c[1]]
    rr2[c[1] + ':MCl_3'] = (rr2['MCl']) * (rr2['third'] * 1) * rr2[c[1]]
    
    rr2[c[1] + ':nr'] = rr2['count'] * rr2[c[1]]

In [11]:
rr1p = pd.merge(rr1, ped, left_on ='Horse', right_on ='Name', how ='left')
rrp = pd.merge(rr2, ped, left_on ='Horse', right_on ='Name', how ='left')

rr3 = rr2.groupby(['Horse']).sum()
rr3f = rr2[['Horse','Datecol']].groupby(['Horse']).min()
rrs = rrp.groupby(['S']).sum()
#rrsc = rrp.groupby(['S']).count()
rrsf = rrp[['S','Datecol']].groupby(['S']).min()
rrd = rrp.groupby(['D']).sum()
#rrdc = rrp.groupby(['D']).count()
rrdf = rrp[['D','Datecol']].groupby(['D']).min()
rrds = rrp.groupby(['DS']).sum()
#rrdsc = rrp.groupby(['DS']).count()
rrdsf = rrp[['DS','Datecol']].groupby(['DS']).min()

In [15]:
colds = sorted(cold, key = lambda x: x[0])

In [13]:
sr.head()

,hip,Price,OBSDate,PRICE,PS,Last Bred,Sort By Dam,Buyer,Price,AlphSort,...,Namef,IFSf,UTf,Damf,Siref,Mf,Yf,ODf,Areaf,Statef
0,143,NaN,2021-06-01,NaN,NaN,NaN,BlueHoliday,Withdrawn,Out,Davies Julie,...,0.0,NaN,0.0,Blue Holiday,Exaggerator,0.0,0.0,0.0,NY,NaN
1,69,NaN,2021-06-01,NaN,NaN,NaN,AintSheClassy,Withdrawn,Out,G. A. Thoroughbred Sales,...,0.0,NaN,0.0,Ain't She Classy,Goldencents,0.0,0.0,0.0,KY,NaN
2,186,NaN,2021-06-01,NaN,NaN,NaN,ChasingtheMoon,Jaime Cruz-Delgado,8000,Envision Equine,...,0.0,NaN,0.0,Chasing the Moon,Upstart,0.0,0.0,0.0,KY,NaN
3,245,NaN,2021-06-01,NaN,NaN,NaN,Decoder,"West Point, L.E.B., Agent",300000,Dixon Gabriel,...,0.0,NaN,0.0,Decoder,Classic Empire,0.0,0.0,0.0,KY,NaN
4,281,NaN,2021-06-01,NaN,PS,NaN,EarlyVintage,Antonio Sano,37500,Davies Julie,...,0.0,NaN,0.0,Early Vintage,Classic Empire,0.0,0.0,0.0,KY,NaN


In [17]:
dx = sr[['OBSDate', 'Siref', 'Damf', 'UTf','hip']]
pdf = pd.DataFrame(columns=['OBSDate', 'Pricef', 'Siref', 's_zms', 'Damf', 'd_zms', 'Damsire', 'ds_zms'])
spdf = pd.DataFrame(columns=['OBSDate', 'Pricef', 'Siref', 's_zms', 'Damf', 'd_zms', 'Damsire', 'ds_zms'])
dpdf = pd.DataFrame(columns=['OBSDate', 'Pricef', 'Siref', 's_zms', 'Damf', 'd_zms', 'Damsire', 'ds_zms'])
dspdf = pd.DataFrame(columns=['OBSDate', 'Pricef', 'Siref', 's_zms', 'Damf', 'd_zms', 'Damsire', 'ds_zms'])
#dfs[cold2['1-2011']]

dx = pd.merge(dx, ped[['Name','S']], left_on ='Damf', right_on ='Name', how ='left')
dx.rename(columns = {'S':'Damsire'}, inplace = True)
dx.drop('Name', axis=1, inplace=True)
#dx['Damsire'] = dx['Damsire'].replace('no data found',np.NaN)

dfs = dict(tuple(dx.groupby('OBSDate')))
i=1

for c in colds:
#    print(dfs[v].isna().sum())
    ###  SIRE
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',
                                        c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Siref', right_on = rr3.index, how ='left')    
    dfs[c[0]].rename(columns = {c[1]+':G1':'s_g1',c[1]+':G2':'s_g2',c[1]+':G3':'s_g3',c[1]+':Al':'s_al',c[1]+':MCl':'s_MCl',
                               c[1]+':G1_2':'s_g1_2',c[1]+':G2_2':'s_g2_2',c[1]+':G3_2':'s_g3_2',c[1]+':Al_2':'s_al_2',c[1]+':MCl_2':'s_MCl_2',
                               c[1]+':G1_3':'s_g1_3',c[1]+':G2_3':'s_g2_3',c[1]+':G3_3':'s_g3_3',c[1]+':Al_3':'s_al_3',c[1]+':MCl_3':'s_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3f[['Datecol']], left_on ='Siref', right_on = rr3f.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'s_frdate'}, inplace = True)
    
#    if spdf.shape[0] > 0:
#        dfs[c[0]] = pd.merge(dfs[c[0]], spdf[['s_zms']], left_on = 'Siref', right_on = spdf.index, how='left')
#        dfs[c[0]].columns = [*dfs[c[0]].columns[:-1], 's_pzms']
    ###  SIRE PROGENY
    dfs[c[0]] = pd.merge(dfs[c[0]], rrs[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',c[1]+':nr',
                                        c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Siref', right_on = rrs.index, how ='left')
    dfs[c[0]].rename(columns = {c[1]+':G1':'sp_g1',c[1]+':G2':'sp_g2',c[1]+':G3':'sp_g3',c[1]+':Al':'sp_al',c[1]+':MCl':'sp_MCl',c[1]+':nr':'spnr',
                               c[1]+':G1_2':'sp_g1_2',c[1]+':G2_2':'sp_g2_2',c[1]+':G3_2':'sp_g3_2',c[1]+':Al_2':'sp_al_2',c[1]+':MCl_2':'sp_MCl_2',
                               c[1]+':G1_3':'sp_g1_3',c[1]+':G2_3':'sp_g2_3',c[1]+':G3_3':'sp_g3_3',c[1]+':Al_3':'sp_al_3',c[1]+':MCl_3':'sp_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rrsf[['Datecol']], left_on ='Siref', right_on = rrsf.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'sp_frdate'}, inplace = True)
    
    ###  DAM
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',
                                        c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Damf', right_on = rr3.index, how ='left')
    dfs[c[0]].rename(columns = {c[1]+':G1':'d_g1',c[1]+':G2':'d_g2',c[1]+':G3':'d_g3',c[1]+':Al':'d_al',c[1]+':MCl':'d_MCl',
                               c[1]+':G1_2':'d_g1_2',c[1]+':G2_2':'d_g2_2',c[1]+':G3_2':'d_g3_2',c[1]+':Al_2':'d_al_2',c[1]+':MCl_2':'d_MCl_2',
                               c[1]+':G1_3':'d_g1_3',c[1]+':G2_3':'d_g2_3',c[1]+':G3_3':'d_g3_3',c[1]+':Al_3':'d_al_3',c[1]+':MCl_3':'d_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3f[['Datecol']], left_on ='Damf', right_on = rr3f.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'d_frdate'}, inplace = True)
    
#    if dpdf.shape[0] > 0:
#        dfs[c[0]] = pd.merge(dfs[c[0]], dpdf[['d_zms']], left_on = 'Damf', right_on = dpdf.index, how='left')
#        dfs[c[0]].columns = [*dfs[c[0]].columns[:-1], 'd_pzms']
    ###  DAM PROGENY
    dfs[c[0]] = pd.merge(dfs[c[0]], rrd[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',c[1]+':nr',
                                        c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Damf', right_on = rrd.index, how ='left')
    dfs[c[0]].rename(columns = {c[1]+':G1':'dp_g1',c[1]+':G2':'dp_g2',c[1]+':G3':'dp_g3',c[1]+':Al':'dp_al',c[1]+':MCl':'dp_MCl',c[1]+':nr':'dpnr',
                               c[1]+':G1_2':'dp_g1_2',c[1]+':G2_2':'dp_g2_2',c[1]+':G3_2':'dp_g3_2',c[1]+':Al_2':'dp_al_2',c[1]+':MCl_2':'dp_MCl_2',
                               c[1]+':G1_3':'dp_g1_3',c[1]+':G2_3':'dp_g2_3',c[1]+':G3_3':'dp_g3_3',c[1]+':Al_3':'dp_al_3',c[1]+':MCl_3':'dp_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rrdf[['Datecol']], left_on ='Damf', right_on = rrdf.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'dp_frdate'}, inplace = True)

    ### DAMSIRE
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',
                                        c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Damsire', right_on = rr3.index, how ='left')
    dfs[c[0]].rename(columns = {c[1]+':G1':'ds_g1',c[1]+':G2':'ds_g2',c[1]+':G3':'ds_g3',c[1]+':Al':'ds_al',c[1]+':MCl':'ds_MCl',
                               c[1]+':G1_2':'ds_g1_2',c[1]+':G2_2':'ds_g2_2',c[1]+':G3_2':'ds_g3_2',c[1]+':Al_2':'ds_al_2',c[1]+':MCl_2':'ds_MCl_2',
                               c[1]+':G1_3':'ds_g1_3',c[1]+':G2_3':'ds_g2_3',c[1]+':G3_3':'ds_g3_3',c[1]+':Al_3':'ds_al_3',c[1]+':MCl_3':'ds_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rr3f[['Datecol']], left_on ='Damsire', right_on = rr3f.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'ds_frdate'}, inplace = True)
    
#    if dspdf.shape[0] > 0:
#        dfs[c[0]] = pd.merge(dfs[c[0]], dspdf[['ds_zms']], left_on = 'Damsire', right_on = dspdf.index, how='left')
#        dfs[c[0]].columns = [*dfs[c[0]].columns[:-1], 'ds_pzms']
    ### DAMSIRE PROGENY
    dfs[c[0]] = pd.merge(dfs[c[0]], rrds[[c[1]+':G1',c[1]+':G2',c[1]+':G3',c[1]+':Al',c[1]+':MCl',c[1]+':nr',
                                         c[1]+':G1_2',c[1]+':G2_2',c[1]+':G3_2',c[1]+':Al_2',c[1]+':MCl_2',
                                        c[1]+':G1_3',c[1]+':G2_3',c[1]+':G3_3',c[1]+':Al_3',c[1]+':MCl_3']], left_on ='Damsire', right_on = rrds.index, how ='left')
    dfs[c[0]].rename(columns = {c[1]+':G1':'dsp_g1',c[1]+':G2':'dsp_g2',c[1]+':G3':'dsp_g3',c[1]+':Al':'dsp_al',c[1]+':MCl':'dsp_MCl',c[1]+':nr':'dspnr',
                               c[1]+':G1_2':'dsp_g1_2',c[1]+':G2_2':'dsp_g2_2',c[1]+':G3_2':'dsp_g3_2',c[1]+':Al_2':'dsp_al_2',c[1]+':MCl_2':'dsp_MCl_2',
                               c[1]+':G1_3':'dsp_g1_3',c[1]+':G2_3':'dsp_g2_3',c[1]+':G3_3':'dsp_g3_3',c[1]+':Al_3':'dsp_al_3',c[1]+':MCl_3':'dsp_MCl_3'}, inplace = True)
    
    dfs[c[0]] = pd.merge(dfs[c[0]], rrdsf[['Datecol']], left_on ='Damsire', right_on = rrdsf.index, how ='left')    
    dfs[c[0]].rename(columns = {'Datecol':'dsp_frdate'}, inplace = True)
    
    dfs[c[0]]['s_nsales'] = dfs[c[0]].groupby(['Siref'])['Siref'].transform('count')
#     dfs[c[0]]['s_sales'] = dfs[c[0]].groupby(['Siref'])['Pricef'].transform('sum')
#     dfs[c[0]]['s_msales'] = dfs[c[0]].groupby(['Siref'])['Pricef'].transform('mean').fillna(0)
#    dfs[c[0]]['s_zms:'+c[1]] = dfs[c[0]][['s_msales']].apply(stats.zscore)
#     dfs[c[0]]['s_zms'] = dfs[c[0]][['s_msales']].apply(stats.zscore)
    
    dfs[c[0]]['d_nsales'] = dfs[c[0]].groupby(['Damf'])['Damf'].transform('count')
#     dfs[c[0]]['d_sales'] = dfs[c[0]].groupby(['Damf'])['Pricef'].transform('sum')
#     dfs[c[0]]['d_msales'] = dfs[c[0]].groupby(['Damf'])['Pricef'].transform('mean').fillna(0)
#    dfs[c[0]]['d_zms:'+c[1]] = dfs[c[0]][['d_msales']].apply(stats.zscore)
#     dfs[c[0]]['d_zms'] = dfs[c[0]][['d_msales']].apply(stats.zscore)
    
    dfs[c[0]]['ds_nsales'] = dfs[c[0]].groupby(['Damsire'])['Damsire'].transform('count')
#     dfs[c[0]]['ds_sales'] = dfs[c[0]].groupby(['Damsire'])['Pricef'].transform('sum')
#     dfs[c[0]]['ds_msales'] = dfs[c[0]].groupby(['Damsire'])['Pricef'].transform('mean').fillna(0)
#    dfs[c[0]]['ds_zms:'+c[1]] = dfs[c[0]][['ds_msales']].apply(stats.zscore)
#     dfs[c[0]]['ds_zms'] = dfs[c[0]][['ds_msales']].apply(stats.zscore)
    
    spdf = dfs[c[0]].drop_duplicates(subset=['Siref'], keep='first')
    spdf.set_index('Siref', inplace=True)
    dpdf = dfs[c[0]].drop_duplicates(subset=['Damf'], keep='first')
    dpdf.set_index('Damf', inplace=True)
    dspdf = dfs[c[0]].drop_duplicates(subset=['Damsire'], keep='first')
    dspdf.set_index('Damsire', inplace=True)
    
#    print(dfs[v].isna().sum())
#    sirel = list(dfs[v]['Siref'].unique())
#    for s in sirel:

dfexp = pd.concat(dfs)

In [18]:
dfJune = dfs[cold2['6-2021']]
dfJune.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\JuneOBS15performance.csv')

In [21]:
dfexp

OBSDate    Pricef             Siref              Damf  \
2011-01-01 0   2011-01-01    9000.0         Unbridled           Nannerl   
           1   2011-01-01   10000.0      Wildcat Heir      Note to Mimi   
           2   2011-01-01    7000.0       Private Vow          Parkdale   
           3   2011-01-01   20000.0           Matty G      Party Ribbon   
           4   2011-01-01    1200.0  With Distinction         Pearlwood   
...                   ...       ...               ...               ...   
2021-03-01 322 2021-03-01    9000.0     Wicked Strong       Holy Roller   
           323 2021-03-01  165000.0        First Dude      Home in Time   
           324 2021-03-01   70000.0    Classic Empire      Honest Cause   
           325 2021-03-01   70000.0      Street Sense        Honey Ride   
           326 2021-03-01  375000.0       Ghostzapper  Hudson River Gal   

                         Damsire  s_g1  s_g2  s_g3  s_al   s_frdate  ...  \
2011-01-01 0        Valid Appeal   4.0   0.0   0.0   2.0 1990-05-05  ...   
           1            Notebook   0.0   0.0   0.0   0.0        NaT  ...   
           2       no data found   0.0   0.0   0.0   0.0 2005-10-29  ...   
           3           Java Gold   0.0   0.0   0.0   0.0 1996-03-02  ...   
           4       no data found   0.0   0.0   0.0   0.0        NaT  ...   
...                          ...   ...   ...   ...   ...        ...  ...   
2021-03-01 322            Pulpit   3.0   2.0   0.0   0.0 2013-09-21  ...   
           323         Came Home   2.0   0.0   2.0   2.0 2009-10-22  ...   
           324  Giant's Causeway   0.0   0.0   0.0   0.0 2007-10-31  ...   
           325        Candy Ride   6.0   2.0   2.0   0.0 2006-07-09  ...   
           326        Dynaformer   8.0   2.0   2.0   2.0 2002-11-16  ...   

                s_msales     s_zms  d_nsales   d_sales  d_msales     d_zms  \
2011-01-01 0      5000.0 -0.329720       1.0    9000.0    9000.0  0.130555   
           1     16812.5  1.122348       1.0   10000.0   10000.0  0.229629   
           2      7000.0 -0.083868       1.0    7000.0    7000.0 -0.067594   
           3     20000.0  1.514176       1.0   20000.0   20000.0  1.220373   
           4      6050.0 -0.200648       1.0    1200.0    1200.0 -0.642226   
...                  ...       ...       ...       ...       ...       ...   
2021-03-01 322    9000.0 -1.156650       1.0    9000.0    9000.0 -0.795349   
           323  165000.0  0.517337       1.0  165000.0  165000.0  0.355737   
           324   65000.0 -0.555731       1.0   70000.0   70000.0 -0.345245   
           325  115000.0 -0.019197       1.0   70000.0   70000.0 -0.345245   
           326  375000.0  2.770781       1.0  375000.0  375000.0  1.905276   

                ds_nsales  ds_sales      ds_msales    ds_zms  
2011-01-01 0          2.0   10700.0    5350.000000 -0.058297  
           1          1.0   10000.0   10000.000000  0.634487  
           2         55.0  468300.0    8514.545455  0.413176  
           3          1.0   20000.0   20000.000000  2.124345  
           4         55.0  468300.0    8514.545455  0.413176  
...                   ...       ...            ...       ...  
2021-03-01 322        2.0   20000.0   10000.000000 -0.795512  
           323        1.0  165000.0  165000.000000  0.674672  
           324        6.0  680000.0  113333.333333  0.184611  
           325        3.0  111000.0   37000.000000 -0.539415  
           326        4.0  654000.0  163500.000000  0.660444  

[26671 rows x 47 columns]

In [ ]:
sll = dfexp[['OBSDate', 'Siref']].apply(tuple, axis=1).tolist()
szms = list()
for s in sll:
    ddf = dfexp[dfexp['OBSDate'] < s[0]]
    ddf2 = ddf[ddf['Siref'] == s[1]]
    if ddf2.shape[0] > 0:
        szms.append(ddf2['s_zms'].iloc[-1])
    else:
        szms.append(0)

In [ ]:
dll = dfexp[['OBSDate', 'Damf']].apply(tuple, axis=1).tolist()
dzms = list()
for d in dll:
    ddf = dfexp[dfexp['OBSDate'] < d[0]]
    ddf2 = ddf[ddf['Damf'] == d[1]]
    if ddf2.shape[0] > 0:
        dzms.append(ddf2['d_zms'].iloc[-1])
    else:
        dzms.append(0)

In [29]:
dsll = dfexp[['OBSDate', 'Damsire']].apply(tuple, axis=1).tolist()
dszms = list()
for ds in dsll:
    ddf = dfexp[dfexp['OBSDate'] < ds[0]]
    ddf2 = ddf[ddf['Damsire'] == ds[1]]
    if ddf2.shape[0] > 0:
        dszms.append(ddf2['ds_zms'].iloc[-1])
    else:
        dszms.append(0)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [50]:
dfexp['s_pzms'] = np.array(szms)
dfexp['d_pzms'] = np.array(dzms)
dfexp['ds_pzms'] = np.array(dszms)

In [51]:
dfexp.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\sddspModel630.csv')

In [32]:
szms

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [17]:
dfs[cold2['3-2021']].groupby(['Siref'])['Siref'].transform('count')

6381     4
6382     3
6383     2
6384     2
6385    11
        ..
6703     1
6704     1
6705     3
6706     2
6707     1
Name: Siref, Length: 327, dtype: int64

In [16]:
dfs[cold2['3-2021']]

,OBSDate,Pricef,Siref,Damf,Damsire
6381,2021-03-01,120000.0,Speightster,Ice for the Lady,Flatter
6382,2021-03-01,200000.0,Outwork,Ida Clark,Speightstown
6383,2021-03-01,90000.0,Tiznow,I'm Breathtaking,NaN
6384,2021-03-01,25000.0,Strong Mandate,Incleveland,Midnight Lute
6385,2021-03-01,42000.0,Klimt,Infinite Hope,Dynaformer
...,...,...,...,...,...
6703,2021-03-01,9000.0,Wicked Strong,Holy Roller,Pulpit
6704,2021-03-01,165000.0,First Dude,Home in Time,Came Home
6705,2021-03-01,70000.0,Classic Empire,Honest Cause,Giant's Causeway
6706,2021-03-01,70000.0,Street Sense,Honey Ride,Candy Ride


In [37]:
pdf.shape[0]

0

In [129]:
tapit = rrpx[rrpx['S'] == 'Tapit'].sort_values(by='Datecol')
rhumb = rrpx[rrpx['D'] == 'Rhumb Line'].sort_values(by='Datecol')

tapit.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\tapit620.csv')
rhumb.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\rhumb620.csv')

In [111]:
rhumb

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,...,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD
112706,08/16/09-R4,Star Of Sapphire,4th,Dmr,6 1/2 f,D,MSW,2,It Tiz,Seriously,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112705,09/07/09-R2,Star Of Sapphire,3rd,Dmr,1 mile,D,MSW,2,Pure Class,Dixie's Sandals,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112704,10/10/09-R2,Star Of Sapphire,4th,SA,1 mile,T,MSW,2,Andina,Warren's Jitterbug,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112703,02/07/10-R7,Star Of Sapphire,6th,GP,7 f,D,MSW,3,Taconic Victory,Cat Be Nimble,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112701,09/04/10-R3,Zazu,2nd,Dmr,5 1/2 f,D,MSW,2,Tales in Excess,Zazu,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112700,10/15/10-R6,Zazu,1st,Hol,6 1/2 f,S,MSW,2,Zazu,Nereid,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112699,11/21/10-R7,Zazu,2nd,Hol,7 f,S,Moccasin Stakes,2,Turbulent Descent,Zazu,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112698,12/11/10-R9,Zazu,5th,Hol,1 1/16 m,S,Hollywood Starlet-G1,2,Turbulent Descent,Big Tiz,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112697,01/15/11-R6,Zazu,2nd,SA,7 f,D,Santa Ynez-G3,3,California Nectar,Zazu,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre
112696,02/05/11-R4,Zazu,1st,SA,1 mile,D,Las Virgenes-G1,3,Zazu,Turbulent Descent,...,Tap Your Heels,Unbridled,Ruby Slippers,Rhumb Line,Mr. Greeley,Gone West,Long Legend,Rose Rhapsody,Pleasant Colony,Rosy Spectre


In [119]:
tapit.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\tapit620.csv')
rhumb.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\rhumb620.csv')

In [122]:
tapit.shape

(2247, 257)

In [127]:
rr1

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,...,6-2019:G2,6-2019:G3,10-2018,10-2018:G1,10-2018:G2,10-2018:G3,10-2019,10-2019:G1,10-2019:G2,10-2019:G3
0,11/04/16-R7,Tamarkuz,1,SA,1 mile,D,Breeders' Cup Dirt Mile-G1,3+,Tamarkuz,Gun Runner,...,0,0,True,1,0,0,True,1,0,0
1,04/18/00-R4,Rocky Bar,1,TuP,3f,D,Allow,2,Rocky Bar,Cyk Katie,...,0,0,True,0,0,0,True,0,0,0
2,10/08/16-R5,Tamarkuz,2,Bel,1 mile,D,Kelso Handicap-G2,3+,Anchor Down,Tamarkuz,...,0,0,True,0,0,0,True,0,0,0
3,10/06/79-R8,Affirmed,1,Bel,1 1/2m,D,Jockey Club Gold Cup,3+,Affirmed,Spectacular Bid,...,0,0,True,0,0,0,True,0,0,0
4,08/27/16-R9,Tamarkuz,2,Sar,7 f,D,Forego-G1,3+,A. P. Indian,Tamarkuz,...,0,0,True,0,0,0,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468842,11/01/20-R6,Wicked Prankster,7th,Lrl,6 f,D,MSW,2,Xtreme Mayhem,Shackqueenking,...,0,0,False,0,0,0,False,0,0,0
468843,10/22/20-R6,Wicked Prankster,8th,Lrl,5 1/2 f,T,MSW,2,Town of Gold,Gallant George,...,0,0,False,0,0,0,False,0,0,0
468844,09/25/20-R6,Wicked Prankster,2nd,Pim,5 f,T,MSW,2,San Antone,Wicked Prankster,...,0,0,False,0,0,0,False,0,0,0
468845,09/07/20-R2,Wicked Prankster,2nd,Lrl,6 f,D,Mcl,2,Passage Of Lines,Wicked Prankster,...,0,0,False,0,0,0,False,0,0,0


In [124]:
rr2

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,...,6-2019:G2,6-2019:G3,10-2018,10-2018:G1,10-2018:G2,10-2018:G3,10-2019,10-2019:G1,10-2019:G2,10-2019:G3
0,11/04/16-R7,Tamarkuz,1,SA,1 mile,D,Breeders' Cup Dirt Mile-G1,3+,Tamarkuz,Gun Runner,...,0,0,True,1,0,0,True,1,0,0
1,04/18/00-R4,Rocky Bar,1,TuP,3f,D,Allow,2,Rocky Bar,Cyk Katie,...,0,0,True,0,0,0,True,0,0,0
2,10/08/16-R5,Tamarkuz,2,Bel,1 mile,D,Kelso Handicap-G2,3+,Anchor Down,Tamarkuz,...,0,0,True,0,0,0,True,0,0,0
3,10/06/79-R8,Affirmed,1,Bel,1 1/2m,D,Jockey Club Gold Cup,3+,Affirmed,Spectacular Bid,...,0,0,True,0,0,0,True,0,0,0
4,08/27/16-R9,Tamarkuz,2,Sar,7 f,D,Forego-G1,3+,A. P. Indian,Tamarkuz,...,0,0,True,0,0,0,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468842,11/01/20-R6,Wicked Prankster,7th,Lrl,6 f,D,MSW,2,Xtreme Mayhem,Shackqueenking,...,0,0,False,0,0,0,False,0,0,0
468843,10/22/20-R6,Wicked Prankster,8th,Lrl,5 1/2 f,T,MSW,2,Town of Gold,Gallant George,...,0,0,False,0,0,0,False,0,0,0
468844,09/25/20-R6,Wicked Prankster,2nd,Pim,5 f,T,MSW,2,San Antone,Wicked Prankster,...,0,0,False,0,0,0,False,0,0,0
468845,09/07/20-R2,Wicked Prankster,2nd,Lrl,6 f,D,Mcl,2,Passage Of Lines,Wicked Prankster,...,0,0,False,0,0,0,False,0,0,0


In [125]:
rr

,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time,Datecol,Grade
0,11/04/16-R7,Tamarkuz,1,SA,1 mile,D,Breeders' Cup Dirt Mile-G1,3+,Tamarkuz,Gun Runner,Accelerate,1:35.72,2016-11-04,G1
1,04/18/00-R4,Rocky Bar,1,TuP,3f,D,Allow,2,Rocky Bar,Cyk Katie,Carson City Red,0:31.80,2000-04-18,ow
2,10/08/16-R5,Tamarkuz,2,Bel,1 mile,D,Kelso Handicap-G2,3+,Anchor Down,Tamarkuz,Ocean Knight,1:32.90,2016-10-08,G2
3,10/06/79-R8,Affirmed,1,Bel,1 1/2m,D,Jockey Club Gold Cup,3+,Affirmed,Spectacular Bid,Coastal,2:27.40,1979-10-06,up
4,08/27/16-R9,Tamarkuz,2,Sar,7 f,D,Forego-G1,3+,A. P. Indian,Tamarkuz,Stallwalkin' Dude,1:20.99,2016-08-27,G1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468842,11/01/20-R6,Wicked Prankster,7th,Lrl,6 f,D,MSW,2,Xtreme Mayhem,Shackqueenking,Rippolino,1:11.86,2020-11-01,SW
468843,10/22/20-R6,Wicked Prankster,8th,Lrl,5 1/2 f,T,MSW,2,Town of Gold,Gallant George,High Limit Room,1:07.63,2020-10-22,SW
468844,09/25/20-R6,Wicked Prankster,2nd,Pim,5 f,T,MSW,2,San Antone,Wicked Prankster,Twice As Smart,0:59.60,2020-09-25,SW
468845,09/07/20-R2,Wicked Prankster,2nd,Lrl,6 f,D,Mcl,2,Passage Of Lines,Wicked Prankster,King Alan,1:11.68,2020-09-07,cl


In [128]:
rrpx = pd.merge(rr, ped, left_on ='Horse', right_on ='Name', how ='left')

In [38]:
rrd.loc['Siberian Fur']

count         6.0
G1            4.0
G2            0.0
G3            0.0
ow            0.0
             ... 
10-2019:G1    2.0
10-2019:G2    0.0
10-2019:G3    0.0
10-2019:Al    0.0
10-2019:nr    4.0
Name: Siberian Fur, Length: 342, dtype: float64

In [39]:
sr[sr['Damf'] == 'Rhumb Line']

,Unnamed: 0,OBSDate,PS,Last Bred,Sort By Dam,Buyer,AlphSort,Foal Date,set,date,...,IFSf,WTf,Damf,Siref,Mf,Yf,ODf,UTf,Areaf,Statef
4770,349,2014-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,Rhumb Line,Tapit,0.0,0.0,0.0,0.0,NaN,NaN


In [2]:
sr['Siref','Damf','UTf']

NameError: name 'sr' is not defined

In [ ]:
dfJune = dfs[cold2['6-2021']]

In [31]:
l2 = dfJune
# lx = lx.merge(prog2, how='left', left_on='Siref', right_on='Name')
# lx = lx.merge(prog2, how='left', left_on='Damf', right_on='Name')
# lx = lx.merge(prog2, how='left', left_on='Damsire', right_on='Name')

# lx = lx.drop_duplicates(keep='first')

# lx['mnprog_x'] = lx['mnprog_x'].fillna(0)
# lx['mnprog_y'] = lx['mnprog_y'].fillna(0)
# lx['mnprog'] = lx['mnprog'].fillna(0)

pd.to_datetime(l2['OBSDate'])

# lx['AuctionMonth2'] = pd.to_datetime(lx['OBSDate']).dt.month
# dummies = pd.get_dummies(lx['AuctionMonth2'])
# lz = pd.concat([lx, dummies], axis=1)
# l2 = lz[lz[3] == 1]
#l2=lz
l2['sgrades'] = l2['s_g1'] + l2['s_g2'] + l2['s_g3'] + l2['s_al']
l2['dgrades'] = l2['d_g1'] + l2['d_g2'] + l2['d_g3'] + l2['d_al'] + l2['d_g1_2'] + l2['d_g2_2'] + l2['d_g3_2'] + l2['d_al_2']+ l2['d_g1_3'] + l2['d_g2_3'] + l2['d_g3_3'] + l2['d_al_3']
l2['dsgrades'] = l2['ds_g1'] + l2['ds_g2'] + l2['ds_g3'] + l2['ds_al'] + l2['ds_g1_2'] + l2['ds_g2_2'] + l2['ds_g3_2'] + l2['ds_al_2'] + l2['ds_g1_3'] + l2['ds_g2_3'] + l2['ds_g3_3']+ l2['ds_al_3']
l2['spgrades'] = l2['sp_g1'] + l2['sp_g2'] + l2['sp_g3'] + l2['sp_al'] + l2['sp_g1_2'] + l2['sp_g2_2'] + l2['sp_g3_2'] + l2['sp_al_2'] + l2['sp_g1_3'] + l2['sp_g2_3'] + l2['sp_g3_3'] + l2['sp_al_3']
l2['dpgrades'] = l2['dp_g1'] + l2['dp_g2'] + l2['dp_g3'] + l2['dp_al'] + l2['dp_g1_2'] + l2['dp_g2_2'] + l2['dp_g3_2'] + l2['dp_al_2'] + l2['dp_g1_3'] + l2['dp_g2_3'] + l2['dp_g3_3'] + l2['dp_al_3']
l2['dspgrades'] = l2['dsp_g1'] + l2['dsp_g2'] + l2['dsp_g3'] + l2['dsp_al'] + l2['dsp_g1_2'] + l2['dsp_g2_2'] + l2['dsp_g3_2']*1 + l2['dsp_al_2'] + l2['dsp_g1_3'] + l2['dsp_g2_3'] + l2['dsp_g3_3'] + l2['dsp_al_3']

l2['spgrades%'] = l2['spgrades'] / (l2['spnr'] ** .5)
l2['spgrades%'] = l2['spgrades%'].replace([np.inf, -np.inf], 0)
l2['dpgrades%'] = l2['dpgrades'] / (l2['dpnr'] ** .5)
l2['dpgrades%'] = l2['dpgrades%'].replace([np.inf, -np.inf], 0)
l2['dspgrades%'] = l2['dspgrades'] / (l2['dspnr'] ** .5)
l2['dspgrades%'] = l2['dspgrades%'].replace([np.inf, -np.inf], 0)

l2['gradex'] = (l2['spgrades%'] + 1) * (l2['dpgrades%'] + 1)

# l2['s_precoc'] = abs((pd.to_datetime(l2['s_frdate']) - pd.to_datetime(l2['sp_frdate'])).dt.days)
# l2['d_precoc'] = abs((pd.to_datetime(l2['d_frdate']) - pd.to_datetime(l2['dp_frdate'])).dt.days)
# l2['ds_precoc'] = abs((pd.to_datetime(l2['ds_frdate']) - pd.to_datetime(l2['dsp_frdate'])).dt.days)

# l2['s_precoc'] = l2['s_precoc'].fillna(l2['s_precoc'].mean())
# l2['d_precoc'] = l2['d_precoc'].fillna(l2['d_precoc'].mean())
# l2['ds_precoc'] = l2['ds_precoc'].fillna(l2['ds_precoc'].mean())

# l2['s_pzms'] = l2['s_pzms'].fillna(0)
# l2['d_pzms'] = l2['d_pzms'].fillna(0)
# l2['ds_pzms'] = l2['ds_pzms'].fillna(0)

# l2['pzmsx'] = ((l2['s_pzms']+1) * (l2['d_pzms'] + 1)) ** 2

l2['spgrades%'] = l2['spgrades%'].fillna(0)
l2['dpgrades%'] = l2['dpgrades%'].fillna(0)
l2['dspgrades%'] = l2['dspgrades%'].fillna(0)

#l2['OBSDate'] = pd.to_datetime(l2['OBSDate']).fillna(dt.date.today()).map(dt.datetime.fromordinal)

TypeError: an integer is required (got type Timestamp)

In [29]:
l3 = l2[['hip','Siref','Damf','OBSDate', 'sgrades', 'spgrades', 'spgrades%', 'spnr',
        'dgrades', 'dpgrades', 'dpgrades%', 'dpnr', 
        'dsgrades', 'dspgrades', 'dspgrades%', 'dspnr']]

In [30]:
l3.head()

,hip,Siref,Damf,OBSDate,sgrades,spgrades,spgrades%,spnr,dgrades,dpgrades,dpgrades%,dpnr,dsgrades,dspgrades,dspgrades%,dspnr
0,143,Exaggerator,Blue Holiday,737942,162,8.0,0.556038,207.0,0.0,36.0,4.213481,73.0,15.0,671.0,20.014252,1124.0
1,69,Goldencents,Ain't She Classy,737942,165,204.0,11.512386,314.0,0.0,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN
2,186,Upstart,Chasing the Moon,737942,57,42.0,3.575277,138.0,0.0,NaN,0.000000,NaN,0.0,15659.0,45.662765,117599.0
3,245,Classic Empire,Decoder,737942,0,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.0,88.0,6.809644,167.0
4,281,Classic Empire,Early Vintage,737942,0,NaN,0.000000,NaN,0.0,16.0,2.785242,33.0,25.0,37.0,2.796937,175.0


In [32]:
l3.shape

(15, 16)

In [33]:
l3.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\OBSJune21sample15.csv')